In [57]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r requirements.txt

# Restart the runtime by ending the process.
exit()

In [6]:
from __future__ import annotations
import os

import ee
import google.auth

import folium
from serving import data
import json
from branca.element import Figure
from folium import plugins
import sys
from importlib import reload
from serving.data import get_input_image_ee, get_varied_labels
from serving.constants import  BUCKET, PROJECT, CROP, SCALE
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.cloud import storage
import logging

In [2]:
current_dir = os.getcwd()
sys.path.append(current_dir+'/serving')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
# Google cloud parameters
project = PROJECT
bucket = BUCKET
location = "us-west2-a"  # @param {type:"string"}

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"
assert bucket, "⚠️ Please provide a Cloud Storage bucket name"
assert not bucket.startswith(
    "gs://"
), f"⚠️ Please remove the gs:// prefix from the bucket name: {bucket}"
assert location, "⚠️ Please provide a Google Cloud location"

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ["GOOGLE_CLOUD_PROJECT"] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

# Initialise goofle earth engine
credentials, _ = google.auth.default()
ee.Initialize(
    credentials.with_quota_project(None),
    project=project,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

Updated property [core/project].


In [4]:
#CONSTANTS
YEAR_START=2017
YEAR_END=2023

SEASON_START=5
SEASON_END=10

CROP=1 #Corn

# True vision params
vis_params = {
    "min": 0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],}

vis_agro = {
     "min": 225,
    "max": 4000,
    "bands": ['B11', 'B8', 'B3']}

In [11]:
reload(data)
from serving.data import get_input_image_ee, get_varied_labels

In [12]:
# Show sample filter

start_year = 2016
start_month = 4
end_month = 7
crop_type = 1 # Corn
county = "Sutter".capitalize()
state_fips = "06"

county_geom = (
    ee.FeatureCollection("TIGER/2018/Counties")
    .filter(ee.Filter.eq("NAME", county))
    .filter(ee.Filter.eq("STATEFP", state_fips))
).geometry()

coords = county_geom.centroid().coordinates().getInfo()

s2_img_start = get_input_image_ee(county, state_fips, crop_type, start_year, start_month)["image"]
s2_img_end = get_input_image_ee(county, state_fips, crop_type, start_year, end_month)["image"]
image_name = get_input_image_ee(county, state_fips, crop_type, start_year, start_month)["image_name"]

In [13]:
image_name

'Sutter_06/2016/4-5'

In [14]:
county, state_fips = image_name.split("/")[0].split("_")

In [16]:
state_fips

'06'

In [11]:
# Create the map
m1 = folium.Map(coords[::-1])

# Add the CDL layer
layer_left = folium.TileLayer(
    tiles=s2_img_start.getMapId(vis_agro)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Cropland Data Layer',
    overlay=True,
    control=True
)

layer_right = folium.TileLayer(
    tiles=s2_img_end.getMapId(vis_agro)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Cropland Data Layer',
    overlay=True,
    control=True,
)

sbs = plugins.SideBySideLayers(layer_left=layer_left, layer_right=layer_right)

layer_left.add_to(m1)
layer_right.add_to(m1)
sbs.add_to(m1)    

# Display the map
m1

In [7]:
"""
Changes in crops can be observable in the satellite images. Once can hope that the colour saturation and temporal change would hold predictive power
"""

'\nChanges in crops can be observable in the satellite images. Once can hope that the colour saturation and temporal change would hold predictive power\n'

In [ ]:
# DATA RETRIVAL

In [17]:
def check_blob_prefix_exists(bucket_name, prefix):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    
    blobs = bucket.list_blobs(prefix=prefix, max_results=1)
    return any(blobs)

def batch_check_blobs(bucket_name, prefixes):
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_prefix = {executor.submit(check_blob_prefix_exists, bucket_name, prefix): prefix for prefix in prefixes}
        results = {}
        for future in as_completed(future_to_prefix):
            prefix = future_to_prefix[future]
            results[prefix] = future.result()
    return results

# Generate all prefixes
bucket_name = bucket
labels_df = get_varied_labels(150)
labels = list(zip(labels_df["county_name"],
             labels_df["state_ansi"],
             labels_df["year"]))
months = [5,7,9]

prefixes = [f'images/{SCALE}/{county}_{fips}/{year}/{month}-{month+1}' 
            for county, fips, year in labels
            for month in months]

# Batch check all prefixes
results = batch_check_blobs(bucket_name, prefixes)

# Generate get_input_img_params based on results
get_input_img_params = [
    {"county": county, "state_fips": fips, "crop": CROP, "year": year, "month": month}
            for county, fips, year in labels
            for month in months
            if not results[f'images/{SCALE}/{county}_{fips}/{year}/{month}-{month+1}']
]

print(f"Number of items to process: {len(get_input_img_params)}")

Number of items to process: 2643


In [ ]:
get_input_img_params_small = get_input_img_params[2:100]

In [29]:
def export_img(image:ee.Image, image_name: str, county_geom: ee.geometry):
    
    image_name = f"{SCALE}/" + image_name
    img_task = ee.batch.Export.image.toCloudStorage(
    image=image,
    description=image_name.replace("/","-"),
    bucket=bucket,
    fileNamePrefix=f"images/{image_name}",
    scale=SCALE,
    region=county_geom,
    fileDimensions = 4*2048,
    skipEmptyTiles = True,
    fileFormat = "GeoTIFF",
    maxPixels = 1e9
    )

    img_task.start()
    logging.info(f"Export task started for: {image_name}")

def apply_get_input_image(params):
    
    return get_input_image_ee(**params)

def unpack_for_export(list_input):
    county, _ = list_input["image_name"].split("/")[0].split("_")
    county_geom = (
    ee.FeatureCollection("TIGER/2018/Counties")
        .filter(ee.Filter.eq("NAME", county.capitalize()))
        .filter(ee.Filter.eq("STATEFP", state_fips))
        ).geometry()
    print(list_input["image_name"])
    return export_img(list_input["image"], list_input["image_name"], county_geom)

In [22]:
from apache_beam.options.pipeline_options import PipelineOptions

# Define your pipeline options
options = PipelineOptions(
    runner='DataflowRunner',  # Specify the runner as 'DataflowRunner'
    project=project,  # Replace with your Google Cloud project ID
    region='us-west2-a',  # Specify the GCP region, e.g., 'us-central1'
    temp_location=f'gs://{BUCKET}/temp',  # GCS bucket for temporary files
    staging_location=f'gs://{BUCKET}/staging',  # GCS bucket for staging files
    job_name='s2-image-top150',  # Give your job a unique name    
    save_main_session=True  # Ensures that all dependencies are available in the workers
)

In [33]:
with beam.Pipeline() as pipeline:
    (
        pipeline
        |"Create parameter sets" >>  beam.Create(get_input_img_params_small)
        | "Sample counties" >> beam.Map(apply_get_input_image)
        | "Export image to GCS bucket" >> beam.Map(unpack_for_export)
    )

2024-10-19 19:23:32,018 - INFO - Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
2024-10-19 19:23:32,171 - INFO - Creating state cache with size 104857600


Sutter_06/2016/9-10


2024-10-19 19:23:34,175 - INFO - Export task started for: 60/Sutter_06/2016/9-10


Sutter_06/2018/5-6


2024-10-19 19:23:35,855 - INFO - Export task started for: 60/Sutter_06/2018/5-6


...

2024-10-19 19:25:57,040 - INFO - Export task started for: 60/Anderson_45/2019/7-8


Anderson_45/2019/9-10


2024-10-19 19:25:58,087 - INFO - Export task started for: 60/Anderson_45/2019/9-10


Anderson_45/2020/5-6


2024-10-19 19:25:59,571 - INFO - Export task started for: 60/Anderson_45/2020/5-6
